<a href="https://colab.research.google.com/github/arieljumba/stock/blob/main/Stockstest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
import dash_table as dtt
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
pd.options.display.float_format = '{0:,.0f}'.format
from dash_table import DataTable, FormatTemplate

#format numbers
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
locale.format_string("%d", 1255000, grouping=True)

#app
app = JupyterDash(external_stylesheets=[dbc.themes.LITERA])
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col([html.H1(id = 'header_title', style = {'text-align':'center','fontSize':'30px','color':'#17202A','justify':'center'})]),
                dbc.Col([dcc.Dropdown(id = 'comp_name_dropdown',options = [{'label':'Apple Inc','value':'AAPL'},
                                                                           {'label':'Bank of America Corporation','value':'BAC'},
                                                                           {'label':'Amazon.com','value':'AMZN'},
                                                                           {'label':'AT&T Inc','value':'T'},
                                                                           {'label':'Alphabet Inc','value':'GOOG'},
                                                                           {'label':'American Airlines','value':'AAL'},
                                                                           {'label':'AstraZeneca PLC.','value':'AZN'},
                                                                           {'label':'DreamWorks Animation SKG Inc.','value':'DWA'}],
                                      style = {'height':'30%'},value = 'AMZN',multi = False)])   
            ],style = {'backgroundColor':'#FBEEE6'}),
            
            html.Br(),
            dbc.Row([dcc.Markdown(id = 'comp_descr')]),
            html.Br(),
            dbc.Row([
                dbc.Col([html.H6('BALANCE SHEET GROWTH(''Million Dollars)')]),
                dbc.Col([html.H6('PROFIT & LOSS STATEMENT ITEMS (''Million Dollars)')]),
                dbc.Col([html.H6('OWNERSHIP STRUCTURE')])
            ],style = {'backgroundColor':'#F2F3F4','height':'50','text-align':'center'}),
            
            dbc.Row([
                dbc.Col([dcc.Graph(id = 'bal_sheet',style = {'width':'95%','backgroundColor':'#F8F9F9'})]),
                dbc.Col([dcc.Graph(id = 'pl_sheet',style = {'width':'95%','backgroundColor':'#F8F9F9'})]),
                dbc.Col([dcc.Graph(id = 'ownership_structure',style = {'width':'95%','backgroundColor':'#F8F9F9'})])          
            ],style = {'backgroundColor':'#F8F9F9','height':'80'}),
            
            html.H6('STOCK ANALYSIS',style = {'backgroundColor':'#F2F3F4','justify':'center'}),
            
            dbc.Row([
                dbc.Col([dcc.Graph(id = 'stock_prices')]),
                dbc.Col([dcc.Graph(id = 'stock_volume')]),
                dbc.Col([])
            ],style = {'backgroundColor':'#F8F9F9','height':'90%','width':'150%'})
            
        ])
    )
    
])

#customize header with company title
@app.callback(
    Output(component_id='header_title', component_property = 'children'),
    Input(component_id = 'comp_name_dropdown',component_property = 'value')
)
def update_header(input_value):
    return 'COMPANY ANALYSIS: {}'.format(input_value)

#update compay description
@app.callback(Output(component_id='comp_descr',component_property='children'),
              Input(component_id='comp_name_dropdown',component_property='value'))
def update_desc(input_value):
    filtered_comp = input_value
    stock = yf.Ticker(filtered_comp)
    stock_desc = stock.get_info()
    stock_desc = stock_desc['longBusinessSummary']
    return stock_desc

#update Balance sheet
@app.callback(Output('bal_sheet','figure'),
              Input('comp_name_dropdown','value'))
def update_bal(input_value):
    filtered_comp = input_value
    stock = yf.Ticker(filtered_comp)
    stock_balsheet = stock.balance_sheet/1000000
    stock_balsheet = pd.DataFrame(stock_balsheet)
    stock_balsheet['Item'] = stock_balsheet.index
    stock_balsheet = stock_balsheet.reset_index(drop=True)
    stock_balsheet.rename(columns = {stock_balsheet.columns[0]:'Current_Year'},inplace=True)
    stock_balsheet.rename(columns = {stock_balsheet.columns[1]:'Previous_Year'},inplace=True)
    stock_balsheet['Annual_Growth'] = stock_balsheet['Current_Year']-stock_balsheet['Previous_Year']
    stock_balsheet = stock_balsheet[['Item','Current_Year','Previous_Year','Annual_Growth']]
    pd.options.display.float_format = '{0:,.0f}'.format

    fig = go.Figure(data=[go.Table(
        header=dict(values=list(stock_balsheet.columns),
                    fill_color='paleturquoise',
                    align='center'),
        cells=dict(values = [stock_balsheet.Item,stock_balsheet.Current_Year,stock_balsheet.Previous_Year,stock_balsheet.Annual_Growth],
                   align = 'left')
    )])
    fig.update_layout(font_size=12,transition_duration=500,
                      margin={'l': 40, 'b': 40, 't': 9, 'r': 0})
    return fig

#Profit & Loss statement
@app.callback(Output('pl_sheet','figure'),
              Input('comp_name_dropdown','value'))
def update_pl(input_value):
    stock = yf.Ticker(input_value)
    stock_pl = stock.financials/1000000
    stock_pl = pd.DataFrame(stock_pl)
    stock_pl['Item'] = stock_pl.index
    stock_pl = stock_pl.reset_index(drop=True)
    stock_pl.rename(columns = {stock_pl.columns[0]:'Current_Year'},inplace=True)
    stock_pl.rename(columns = {stock_pl.columns[1]:'Previous_Year'},inplace=True)
    stock_pl['Annual_Growth'] = stock_pl['Current_Year']-stock_pl['Previous_Year']
    stock_pl = stock_pl[['Item','Current_Year','Previous_Year','Annual_Growth']]
    

    fig = go.Figure(data=[go.Table(
        header=dict(values=list(stock_pl.columns),
                    fill_color='paleturquoise',
                    align='center'),
    cells=dict(values = [stock_pl.Item,stock_pl.Current_Year,stock_pl.Previous_Year,stock_pl.Annual_Growth],
                align = 'left')
    )])
    fig.update_layout(font_size=12,transition_duration=500,
                      margin={'l': 40, 'b': 40, 't': 9, 'r': 0})
    return fig


#Shareholding table
@app.callback(Output('ownership_structure','figure'),
              Input('comp_name_dropdown','value'))
def update_own(input_value):
    stock = yf.Ticker(input_value)
    stock_ownership = stock.get_major_holders()
    stock_ownership = pd.DataFrame(stock_ownership)
    stock_ownership.columns = [['a','b']]
    fig = go.Figure(data = [go.Table(
        header = dict(values = ['Shareholding','Desc'],
                      fill_color = 'paleturquoise',
                      align = 'center'),
        cells = dict(values = [stock_ownership.a,stock_ownership.b],
                     align = 'left'))])
    fig.update_layout(font_size=12,transition_duration=500,
                      margin={'l': 40, 'b': 5, 't': 9, 'r': 0}) 
    return fig

#Update Stock Prices
@app.callback(Output('stock_prices','figure'),
              Output('stock_volume','figure'),
              Input('comp_name_dropdown','value'))

def update_stock(input_value):
    stock = yf.Ticker(input_value)
    stock_prices = stock.history(start = '2021-01-01')
    stock_prices = pd.DataFrame(stock_prices)
    stock_prices['Date'] = stock_prices.index
    fig = px.line(stock_prices,y = ['Close','Open','High','Low'],title='Daily Stock Prices')
    fig.update_xaxes(dtick = 'M1',
                     tickformat = '%b\n%Y',
                     ticklabelmode = 'period')
    
    fig2 = px.bar(x= stock_prices['Date'], y = stock_prices['Volume'],title = 'Volume Traded')
    fig2.update_xaxes(dtick = 'M1',
                     tickformat = '%b\n%Y',
                     ticklabelmode = 'period')
    return fig,fig2

app.run_server()

ModuleNotFoundError: ignored